# Filipino Family Income and Expenditure: Regional Average


# 1. Import Python packages

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re

# 2. Loading and Categorizing the Dataset

In [3]:
data = pd.read_csv('Family Income and Expenditure.csv')

Categorizing the data into: expenditures, income, household head data, number of appliances, property data, and family composition

In [5]:
from data_utils import _expenditures_data, _income_data, _householdhead_data, _appliances_data, _property_information, _family_composition

expenditures_data = _expenditures_data(data)
income_data = _income_data(data)
householdhead_data = _householdhead_data(data)
appliances_data = _appliances_data(data)
property_data = _property_information(data)
family_composition_data = _family_composition(data)

# Checking if all of the columns have been categorized 
collection = expenditures_data +  income_data + appliances_data +  householdhead_data + family_composition_data +  property_data
missing = [element for element in data.columns if element not in collection]
print(missing)


['Region']


# 3. Exploring the Dataset: Regional Averages

In this section, we will perform data grouping based on regions. Grouping the data per region allows us to explore regional disparities, identify unique economic conditions, and understand how family income and expenditures vary across different parts of the country. We seek to answer the following questions:

1. What are the regions with the highest and lowest average family income?
2. How do family expenditures differ across regions?
3. What are the major expense categories for families in different regions?
4. Is there a correlation between family income and certain types of expenditures?
5. Are there any significant outliers in the data that indicate unique economic conditions in certain regions?

In [6]:
from ipywidgets import interact, fixed
from regional_utils import bar_chart_regional_average

c:\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## 3.1 

We will first visualize the total household income and various expenditures per region. The lowest and highest values are denoted by red and green, respectively. 

In [7]:
keys = ['Total Household Income'] + expenditures_data
interact(bar_chart_regional_average, data=fixed(data), key=keys)


interactive(children=(Dropdown(description='key', options=('Total Household Income', 'Total Food Expenditure',…

<function regional_utils.bar_chart_regional_average(data, key: list)>

In [11]:
def region_with_max_value(data, key):
    """Return the region with the highest average value for the given data column."""
    regional_averages_data = data.groupby("Region").mean()
    print(f'The region with the highest average {key} is {regional_averages_data[key].idxmax()}')
    return regional_averages_data[key].idxmax()


def region_with_min_value(data, key):
    """Return the region with the lowest average value for the given data column."""
    regional_averages_data = data.groupby("Region").mean()
    print(f'The region with the lowest average {key} is {regional_averages_data[key].idxmin()}')
    return regional_averages_data[key].idxmin()

In [ ]:
for key in 

NCR has the highest average total household income and average total food expenditures.


Meanwhile, ARMM had the lowest total household income but not the lowest total household expenditures which is held by Region X - Northern Mindanao.

It would be interesting to see how income and the various expenditures correlate. According to, lower income households tend to consume more staple foods such as rice, grains, and bread, while middle class individuals allocate a higher proportion of their income to meat and vegetables.

In [8]:
from regional_utils import regional_average_dependence
interact(regional_average_dependence, data=fixed(data), key=expenditures_data)

interactive(children=(Dropdown(description='key', options=('Total Food Expenditure', 'Bread and Cereals Expend…

<function regional_utils.regional_average_dependence(data, key)>

# Choropleth map

In [7]:
import geopandas as gpd
from regional_utils import get_string_inside_parenthesis, make_map_text

In [8]:
regions = gpd.GeoDataFrame.from_file('map/ph-regions-2015.shp')
regions.REGION = regions.REGION.apply(get_string_inside_parenthesis)

psgg_code = pd.read_csv('map/psgg_codes.csv', dtype=object)
map_names = psgg_code.loc[:, ['psgg_code', 'region']]
map_names['region'] = map_names['region'].apply(lambda x: make_map_text(x))
map_names.set_index('psgg_code', inplace=True, drop=True)
expenditures_income_data = expenditures_data.append('Total Household Income')

# Removing NIR in the regions shapes
regions_clean = regions.drop(regions.index[-1]) # Drop the last row
regional_averages = data.groupby("Region")[expenditures_data].mean().reset_index()

regional_averages["Region"] = [entry.split()[0] for entry in regional_averages["Region"]] # Representing the regions with numbers
regional_averages.loc[6, "Region"] = "IV-A" # Match the entry in regions_clean
regional_averages.loc[7, "Region"] = "IV-B" # match the entry in regions_clean
regional_averages.loc[2, "Region"] = "XIII" # match the entry in regions_clean

# Adding regional averages to the regions_clean dataframe
merged_df = pd.merge(regions_clean, regional_averages, left_on='REGION', right_on='Region', how='left')
region_order = merged_df['Region']

In [9]:
def choropleth(merged_df, key):
    plt.style.use('ggplot')
    fig, ax = plt.subplots(figsize=(10, 10))

    merged_df.plot(ax=ax, cmap='viridis', column =key, linewidth=1, legend=True)

    for i, point in merged_df.iterrows():
        point_centroid = point.geometry.centroid
        reg_n = region_order[i]
        ax.text(s=reg_n, x=point_centroid.x, y=point_centroid.y, fontsize='large')

    ax.set_title('PH Administrative Regions (Present)', fontfamily='helvetica', fontsize=20)
    ax.set_axis_off()


In [10]:
interact(choropleth, merged_df=fixed(merged_df), key=expenditures_data)

interactive(children=(Dropdown(description='key', options=('Total Food Expenditure', 'Bread and Cereals Expend…

<function __main__.choropleth(merged_df, key)>